In [1]:
import torch 
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Prepare Data

BREAST CANCER DATASET

In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [3]:
X.shape

(569, 30)

In [4]:
n_samples, n_features = X.shape

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)

In [6]:
scaler = StandardScaler()

In [7]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [8]:
X_train_tensor = torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test_scaled.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [9]:
y_train_resized = y_train_tensor.view(y_train_tensor.shape[0], 1)
y_test_resized = y_test_tensor.view(y_test_tensor.shape[0], 1)

MODEL BUILD

In [10]:
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = LogisticRegression(n_features)

# loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# training loop
num_epochs = 100
for epoch in range(num_epochs):
    #forward pass and loss
    y_pred = model(X_train_tensor)
    loss = criterion(y_pred, y_train_resized)
    
    #backward pass
    loss.backward()
    
    #update weights
    optimizer.step()
    
    #zero gradients
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_predicted_cls = y_pred.round()
    acc = y_predicted_cls.eq(y_test_resized).sum() / float(y_test_resized.shape[0])
    print(f'accuracy: {acc:.4f}')

epoch: 10, loss = 0.4970
epoch: 20, loss = 0.4176
epoch: 30, loss = 0.3661
epoch: 40, loss = 0.3298
epoch: 50, loss = 0.3026
epoch: 60, loss = 0.2813
epoch: 70, loss = 0.2640
epoch: 80, loss = 0.2497
epoch: 90, loss = 0.2376
epoch: 100, loss = 0.2271
accuracy: 0.9123
